In [8]:
#ID3 Algorithm

In [123]:
import math
import random

In [144]:
#Make different subsets of data

#Make Decision Tree Model



#REQUIRES THAT LABEL KEY IN DATA POINT IS CALLED "LABEL"
class DecisionTreeNode():
    
    def __init__(self, child_node1, threshold, child_node2, predict, feat):
        self.child1 = child_node1
        self.threshold = threshold
        self.child2 = child_node2
        self.predict = predict
        self.feat = feat
        
    def goto(self, val):
        
        if self.predict is not None:
            return self.predict
        
        if val[self.feat] > self.threshold:
            return self.child1.goto(val)
        else:
            return self.child2.goto(val)
        



def putting_it_all_together(training, max_depth):
    
    if len(training) == 0:
        return
    
    thresholds = []
    
    sorted_features = []
    keys = list(training[0].keys())
    for k in keys:
        sorted_features.append([])
        thresholds.append([])
    for t in training:
        for i in range(len(keys)):
            if keys[i] == 'label':
                continue
            sorted_features[i].append(t[keys[i]])
    
    
    for i in range(len(keys)):
        sorted_features[i].sort()
        
        for j in range(len(sorted_features[i])-1):
            thresholds[i].append((sorted_features[i][j]+sorted_features[i][j+1])/2)
        
        
    print(thresholds)
 
    parent_node = DecisionTreeNode(None, None, None, None, None)
    
    if max_depth == None:
        recursion(training, thresholds, 1, parent_node, float("inf"), 0)
    else:
        recursion(training, thresholds, 1, parent_node, max_depth, 0)
    
    return parent_node
    
    
    
    
    
def recursion(parent_training_subset, thresholds, parent_entropy, cur_node, max_depth, cur_depth):
       

    check_if_good = get_prob_of_splits(parent_training_subset)
    if 1 in check_if_good:
        cur_node.predict = parent_training_subset[0]['label']
        return
          
        
    max_info_gain = -float("inf")
    max_thresh = 0
    max_feat = 0
    
    #thresholds1 = copy.deepcopy(thresholds)
    #thresholds2 = copy.deepcopy(thresholds)

    predict1 = False
    predict2 = False
    
    
    keys = list(parent_training_subset[0].keys())
    for i in range(len(thresholds)):
        for j in range(len(thresholds[i])):
        
            #can I speed this up???
            split1_ = []
            split2_ = []

            for p in parent_training_subset:
                if p[keys[i]] > thresholds[i][j]:
                    split1_.append(p)
                else:
                    split2_.append(p)
            
            prob1_li = get_prob_of_splits(split1_)
            prob2_li = get_prob_of_splits(split2_)
            val = thresholds[i][j]
            
            #if 1 in prob1_li:
            #    print("1: ", prob1_li)
            #    del thresholds1[i][j]
            #elif 1 in prob2_li:
            #    print("2: ", prob2_li)
            #    del thresholds2[i][j]
                    
            info_gain = parent_entropy-(entropy(prob1_li)*(len(split1_)/len(parent_training_subset)) +
                                            entropy(prob2_li)*(len(split2_)/len(parent_training_subset)))

            
            if info_gain > max_info_gain:
                max_info_gain = info_gain
                max_feat = i
                max_thresh = val
                
    
    split1 = []
    split2 = []
    for p in parent_training_subset:
        if p[keys[max_feat]] > max_thresh:
            split1.append(p)
        else:
            split2.append(p)
            
    #I don't scale this????? entropy1*len(split1)/len(parent)?
    entropy1 = entropy(get_prob_of_splits(split1))
    entropy2 = entropy(get_prob_of_splits(split2))
    
    print(parent_training_subset, thresholds, parent_entropy)
    
    d1 = DecisionTreeNode(None, None, None, None, None)
    d2 = DecisionTreeNode(None, None, None, None, None)
    cur_node.threshold = max_thresh
    cur_node.child1 = d1
    cur_node.child2 = d2
    cur_node.feat = keys[max_feat]

    
    if entropy1 == 0:
        cur_node.child1.predict = split1[0]['label']
    elif (cur_depth+1) >= max_depth:
        cur_node.child1.predict = most_common_label(split1)
    else:
        recursion(split1, thresholds, entropy1, d1, max_depth, cur_depth+1)
        print("left")
        
    if entropy2 == 0:
        cur_node.child2.predict = split2[0]['label']
    elif (cur_depth+1) >= max_depth:
        cur_node.child2.predict = most_common_label(split2)
    else:
        recursion(split2, thresholds, entropy2, d2, max_depth, cur_depth+1)
        print("right")
        
    
    
def most_common_label(t_set):
    occurences = {}
    for t in t_set:
        if t["label"] not in occurences:
            occurences[t["label"]] = 0
        else:
            occurences[t["label"]] += 1
        
    return max(occurences, key=occurences.get)

def most_common_label_basic(t_set):
    occurences = {}
    for t in t_set:
        if t not in occurences:
            occurences[t] = 0
        else:
            occurences[t] += 1
        
    return max(occurences, key=occurences.get)



def get_prob_of_splits(split):
    dict_values = {}
    for s in split:
        if s['label'] not in dict_values:
            dict_values[s['label']] = 1
        else:
            dict_values[s['label']] += 1
    
    
    
    #number of occurences in each class/num of all occurences
    return [v/len(split) for v in dict_values.values()]


#probability of each class in each split
def entropy(P):
    s = 0
    for p in P:
        s += p*math.log(p, 2)
        
    return -1*s

In [136]:
#case 1
training_set = []

putting_it_all_together(training_set, None)



#case 2
#wait.... label will get counted as a feature...
training_set = [{'label' : 0, 'hi':5}, {'label' : 1, 'hi':2}, {'label': 3, 'hi':6}, {'label': 2, 'hi':0}]

DTree = putting_it_all_together(training_set, 2)

print()
print("Max Depth Decreased")
print()

SmallerDTree = putting_it_all_together(training_set, 1)

[[], [1.0, 3.5, 5.5]]
[{'label': 0, 'hi': 5}, {'label': 1, 'hi': 2}, {'label': 3, 'hi': 6}, {'label': 2, 'hi': 0}] [[], [1.0, 3.5, 5.5]] 1
[{'label': 0, 'hi': 5}, {'label': 3, 'hi': 6}] [[], [1.0, 3.5, 5.5]] 1.0
left
[{'label': 1, 'hi': 2}, {'label': 2, 'hi': 0}] [[], [1.0, 3.5, 5.5]] 1.0
right

Max Depth Decreased

[[], [1.0, 3.5, 5.5]]
[{'label': 0, 'hi': 5}, {'label': 1, 'hi': 2}, {'label': 3, 'hi': 6}, {'label': 2, 'hi': 0}] [[], [1.0, 3.5, 5.5]] 1


In [137]:
for t in training_set:
    print(DTree.goto(t), SmallerDTree.goto(t))

0 0
1 1
3 0
2 1


In [148]:
#Make many decision trees 
#sample training with replacement


class RandomForest():
    
    def __init__(self):
        self.decision_trees = []
        
    def majority_vote(self, val):
        votes = [v.goto(val) for v in self.decision_trees]
        
        return most_common_label_basic(votes)


def random_forest_many_trees(training, max_depth, training_size, num_decision_trees):
    
    forest = RandomForest()
    
    for _ in range(num_decision_trees):
    
        #same training set, sample with replacement
        cur_training_set = []
        for i in range(training_size):
            rand_ind = random.randint(0, len(training)-1)
            cur_training_set.append(training[rand_ind])
    
        print(cur_training_set)
        forest.decision_trees.append(putting_it_all_together(cur_training_set, max_depth))
        
    return forest


In [153]:
r1 = random_forest_many_trees(training_set, None, 2, 5)

[{'label': 3, 'hi': 6}, {'label': 3, 'hi': 6}]
[[], [6.0]]
[{'label': 2, 'hi': 0}, {'label': 1, 'hi': 2}]
[[], [1.0]]
[{'label': 2, 'hi': 0}, {'label': 1, 'hi': 2}] [[], [1.0]] 1
[{'label': 0, 'hi': 5}, {'label': 3, 'hi': 6}]
[[], [5.5]]
[{'label': 0, 'hi': 5}, {'label': 3, 'hi': 6}] [[], [5.5]] 1
[{'label': 1, 'hi': 2}, {'label': 0, 'hi': 5}]
[[], [3.5]]
[{'label': 1, 'hi': 2}, {'label': 0, 'hi': 5}] [[], [3.5]] 1
[{'label': 3, 'hi': 6}, {'label': 2, 'hi': 0}]
[[], [3.0]]
[{'label': 3, 'hi': 6}, {'label': 2, 'hi': 0}] [[], [3.0]] 1


In [152]:
for t in training_set:
    print(r1.majority_vote(t))
    
print()
print("Actual:")
for a in training_set:
    print(a["label"])

0
1
3
1

Actual:
0
1
3
2
